In [1]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt

In [2]:
#select image and rotational offset
path = r"3dTo2d\totalUncertancy\2_+00_+00_angle.png"
img = cv2.imread(path, 0)
dim, _ = img.shape[:2]
box_dim = LR.BOX_DIM
offset = 5
flip = False
prim_rot = 0
secd_rot = 0


In [3]:
#create back-projection for image
def createBox(img):
  line_coords = LR.createCoordsForLoadedImageFrom3d(img, flip, dim)
  world_coords = LR.performBresenham3D(line_coords, LR.DETECTOR_TO_PATIENT, LR.SRC_TO_PATIENT, box_dim)
  box = LR.savePointsWithinSquare(world_coords, box_dim)
  #box_rot = LR.rotatePrimSecond(box, prim_rot, secd_rot)
  #box_rot = np.where(box_rot > 0.5, 1, 0)
  return box #, box_rot

#box , box_rot = createBox(img)
box = createBox(img)


In [4]:
#create slightly rotated boxes
rotations = [(offset, 0), (-offset, 0),
             (0, offset), (0, -offset), 
             (offset, offset), (-offset, offset),
             (offset, -offset), (-offset, -offset)]

boxes_rot = [box]
for i in rotations:
  temp_rot = LR.rotatePrimSecond(np.copy(box), i[0], i[1])
  temp_rot = np.where(temp_rot>0.5,1,0)
  boxes_rot.append(temp_rot)
  print("rotated", i[0],i[1])


rotated 5 0
rotated -5 0
rotated 0 5
rotated 0 -5
rotated 5 5
rotated -5 5
rotated 5 -5
rotated -5 -5


In [5]:
len(rotations)

8

In [6]:
print(len(boxes_rot))
print(len(rotations[0]))

9
2


In [7]:
# Save box projections for each image as numpy arrays
name = "_-00_-00_"
np.save(r"model_uncertainty\rotated\offset_5\R" + name + str(offset) + "_" +
          "00",boxes_rot[0])
for i in range(1,len(boxes_rot)):
  np.save(r"model_uncertainty\rotated\offset_5\R" + name + str(offset) + "_" +
          str(rotations[i-1][0]) + str(rotations[i-1][1]), boxes_rot[i])


In [8]:
import collections
print(np.count_nonzero(boxes_rot[0] == 1))
print(np.count_nonzero(boxes_rot[0] == 2))
print(np.count_nonzero(boxes_rot[1] == 1))
print(np.count_nonzero(boxes_rot[1] == 2))

15907829
0
15414030
0


In [9]:
#render box(es)
#x, y, z = LR.boxToWorldCorrds(boxes_rot[0], box_dim)


In [10]:
#mlab.points3d(x, y, z,
#              mode="cube", color=(1, 0, 0.5), scale_factor=1, opacity=1)
#mlab.show()
